In [1]:
using Revise
using Chemfiles
using contacto
using StaticArrays

in_trj = Trajectory("/home/german/labo/contacto/aux/1mtn.pdb")
in_frm = read(in_trj)
in_top = Topology(in_frm)
coords = round.(positions(in_frm), digits = 3)
natoms = size(coords)[2]

┌ Warning: `finalizer(o, f::Function)` is deprecated, use `finalizer(f, o)` instead.
│   caller = Type at Chemfiles.jl:35 [inlined]
└ @ Core /home/german/.julia/packages/Chemfiles/MzvVI/src/Chemfiles.jl:35
┌ Warning: `assert` is deprecated, use `@assert` instead.
│   caller = read(::Trajectory) at Trajectory.jl:36
└ @ Chemfiles /home/german/.julia/packages/Chemfiles/MzvVI/src/Trajectory.jl:36
┌ Warning: `finalizer(o, f::Function)` is deprecated, use `finalizer(f, o)` instead.
│   caller = Type at Chemfiles.jl:108 [inlined]
└ @ Core /home/german/.julia/packages/Chemfiles/MzvVI/src/Chemfiles.jl:108
┌ Warning: `assert` is deprecated, use `@assert` instead.
│   caller = ip:0x0
└ @ Core :-1
┌ Warning: `finalizer(o, f::Function)` is deprecated, use `finalizer(f, o)` instead.
│   caller = Type at Chemfiles.jl:51 [inlined]
└ @ Core /home/german/.julia/packages/Chemfiles/MzvVI/src/Chemfiles.jl:51
┌ Warning: `unsafe_wrap(T, pointer, dims, own)` is deprecated, use `unsafe_wrap(T, pointer, dims, o

1989

In [2]:
cutoff = round(5., digits = 3)
resolution = .1
t_xyz = transpose(coords)

srt_idx_t_xyz = hcat(sortperm(t_xyz[:, 1]), sortperm(t_xyz[:, 2]), sortperm(t_xyz[:, 3]));

srt_x_t_xyz = t_xyz[srt_idx_t_xyz[:, 1], 1]
srt_y_t_xyz = t_xyz[srt_idx_t_xyz[:, 2], 2]
srt_z_t_xyz = t_xyz[srt_idx_t_xyz[:, 3], 3]


x_min = srt_x_t_xyz[1]
x_max = srt_x_t_xyz[end]
y_min = srt_y_t_xyz[1]
y_max = srt_y_t_xyz[end]
z_min = srt_z_t_xyz[1]
z_max = srt_z_t_xyz[end]

espacio = [ x_max - x_min ; y_max - y_min ; z_max - z_min ]

ctro = round.([ x_min + espacio[1]/2, y_min + espacio[2]/2,  z_min + espacio[3]/2 ], digits = 3)

ncells = convert(Int64, ceil(maximum(espacio) / cutoff))

dim = ncells * cutoff

half_dim = round(dim / 2, digits = 3)

sa = contacto.Voxel(ctro, half_dim, 0)

contacto.Voxel([-10.285, 26.147, -25.168], [-37.785 -37.785 … 17.215 17.215; -1.353 -1.353 … 53.647 53.647; -52.668 2.332 … -52.668 2.332], 27.5, 0, contacto.Voxel(#undef, #undef, 6.90175195417325e-310, 0, #undef, #undef), contacto.Voxel[Voxel(#undef, #undef, 6.90175e-310, 0, #undef, #undef), Voxel(#undef, #undef, 6.90175e-310, 0, #undef, #undef), Voxel(#undef, #undef, 6.90175e-310, 0, #undef, #undef), Voxel(#undef, #undef, 6.90175e-310, 0, #undef, #undef), Voxel(#undef, #undef, 6.90175e-310, 0, #undef, #undef), Voxel(#undef, #undef, 6.90175e-310, 0, #undef, #undef), Voxel(#undef, #undef, 6.90175e-310, 0, #undef, #undef), Voxel(#undef, #undef, 6.90175e-310, 0, #undef, #undef)])

nlevels = convert(Int64, floor(log2(ncells)))
nnodes = sum([ 2^x for x in 1:nlevels ]) ; resto = (ncells - 2^nlevels) * 2 ; nnodes += resto
nodes = Array{Int64}(undef, nnodes+1);

tmp_cell_list = Array{Array{Int64, 1}, 1}(undef, nnodes)
nodes = Array{Array{Int64, 1}, 1}(undef, nlevels+1)
#nodes = Array{Int64, 1}(undef, nnodes+1)
nodes[1] = [ncells]

let i = 1
for i = 1:nnodes
    if nodes[i] != 1
        lh_node = convert(Int64, ceil(nodes[i] / 2))
        rh_node = nodes[i] - lh_node
        
        push!(nodes, lh_node)
        push!(nodes, rh_node)
    end
end
    deleteat!(nodes, 1)
end

In [80]:
nlevels = convert(Int64, floor(log2(ncells)))
nnodes = sum([ 2^x for x in 1:(nlevels) ]) ; resto = (ncells - 2^(nlevels)) * 2 ; nnodes += resto
nodes = Array{Int64}(undef, nnodes+1);

tmp_cell_list = Array{Array{Int64, 1}, 1}(undef, nnodes)
nodes_anchos = Array{Array{Int64, 1}, 1}(undef, nlevels+1)
nodes_anchos[1] = [ncells]
nodes_ranges = Array{Array{Tuple{Int64,Int64}, 1}, 1}(undef, nlevels+1)
nodes_ranges[1] = [(0, ncells)]


x_lh_bound = 1 ; y_lh_bound = 1 ; z_lh_bound = 1
x_md_bound = natoms ; y_md_bound = natoms ; z_md_bound = natoms
x_rh_bound = natoms ; y_rh_bound = natoms ; z_rh_bound = natoms

let i = 1, j = 1; for i = 1:nlevels
    k = i + 1
    nodes_anchos[k] = []
    nodes_ranges[k] = []
    prev = 0
    for j = 1:2^(i-1)
        
        lh_node_ancho = convert(Int64, ceil(nodes_anchos[i][j] / 2))
        rh_node_ancho = nodes_anchos[i][j] - lh_node_ancho
        push!(nodes_anchos[k], lh_node_ancho)
        push!(nodes_anchos[k], rh_node_ancho)
        
        rango = nodes_ranges[i][j][2] - nodes_ranges[i][j][1]
        lh_node_rango = convert(Int64, ceil(rango / 2)) + prev
        rh_node_rango = rango + prev
        push!(nodes_ranges[k], (prev, lh_node_rango))
        push!(nodes_ranges[k], (lh_node_rango, rh_node_rango))
        prev = rh_node_rango
    end
end
end

In [24]:
nodes_indices = Array{Array{Tuple{Int64,Int64}, 1}, 1}(undef, nlevels+1)
nodes_indices[1] = [(1, natoms)]

1-element Array{Tuple{Int64,Int64},1}:
 (1, 1989)

In [8]:
i = 1
k = i + 1
j = 1

x_md_bound = searchsortedlast(srt_x_t_xyz[x_lh_bound:x_md_bound],
    (x_min + cutoff * nodes[k][j]))
y_md_bound = searchsortedlast(srt_y_t_xyz[y_lh_bound:y_md_bound],
    (y_min + cutoff * nodes[k][j]))
z_md_bound = searchsortedlast(srt_z_t_xyz[z_lh_bound:z_md_bound],
    (z_min + cutoff * nodes[k][j]))

1901

In [26]:
nodes_indicies[1][]

(1, 1989)

In [47]:
x_cell_list = Array{Array{Int64, 1}, 1}(undef, ncells)
y_cell_list = Array{Array{Int64, 1}, 1}(undef, ncells)
z_cell_list = Array{Array{Int64, 1}, 1}(undef, ncells);

745

In [22]:
srt_idx_t_xyz[x_lh_bound:x_rh_bound, 1]
srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2]
srt_idx_t_xyz[z_lh_bound:z_rh_bound, 3]

1901-element Array{Int64,1}:
 1731
 1729
 1730
 1728
 1726
 1727
 1776
 1725
 1778
 1723
 1713
 1774
 1773
    ⋮
  953
   72
   73
  834
  950
  872
  859
  973
  860
  969
  933
  835

In [38]:
idx_t_xyz[x_lh_bound:x_rh_bound, 1]
idx_t_xyz[y_lh_bound:y_rh_bound, 2]
idx_t_xyz[z_lh_bound:z_rh_bound, 3]

745-element Array{Int64,1}:
  241
  242
  240
  238
  236
  237
  239
  234
  235
  233
  232
  231
  228
    ⋮
 1775
 1749
 1037
 1033
 1748
  491
 1800
 1772
  941
 1798
 1843
  959

In [48]:
lh_srt_idx_t_xyz_x = srt_idx_t_xyz[x_lh_bound:x_md_bound-1, 1]
lh_srt_idx_t_xyz_y = srt_idx_t_xyz[y_lh_bound:y_md_bound-1, 2]
lh_srt_idx_t_xyz_z = srt_idx_t_xyz[z_lh_bound:z_md_bound-1, 3]

rh_srt_idx_t_xyz_x = srt_idx_t_xyz[x_md_bound:x_rh_bound, 1]
rh_srt_idx_t_xyz_y = srt_idx_t_xyz[y_md_bound:y_rh_bound, 2]
rh_srt_idx_t_xyz_z = srt_idx_t_xyz[z_md_bound:z_rh_bound, 3];

In [100]:
octant_0 = Array{Int64}(undef)
let i = 1; for i = 1:length(srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2])
    
    idx_xy = searchsortedlast(srt_idx_t_xyz[x_lh_bound:x_rh_bound, 1],
    srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2][i])
    
    if idx_xy != 0
        if  srt_idx_t_xyz[x_lh_bound:x_rh_bound, 1][idx_xy] ==
        srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2][i]
        
            idx_zy = searchsortedlast(srt_idx_t_xyz[z_lh_bound:z_rh_bound, 3],
                srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2][i])
            if idx_zy != 0
                if  srt_idx_t_xyz[z_lh_bound:z_rh_bound, 3][idx_zy] ==
                srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2][i]
                
                    push!(octant_0, i)
                end
            end
        end
    end
end 
end

4-element Array{Array{Int64,1},1}:
 [11]                    
 [6, 5]                  
 [3, 3, 3, 2]            
 [2, 1, 2, 1, 2, 1, 1, 1]

4-element Array{Array{Int64,1},1}:
 [11]                    
 [6, 5]                  
 [3, 3, 3, 2]            
 [2, 1, 2, 1, 2, 1, 1, 1]